# Imports

In [3]:
# Import standard libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# API
import requests
import json
import os

# Date
import datetime as datetime
from sodapy import Socrata

## API Call: NYC Parking Tickets

In [4]:
# Export sodapy token

socrata_domain = 'data.cityofnewyork.us'
socrata_dataset_identifier = 'nc67-uf89'
socrata_token = os.environ.get("SODAPY_APPTOKEN")

In [5]:
client = Socrata(socrata_domain, socrata_token)
print("Domain: {domain:}\nSession: {session:}\nURI Prefix: {uri_prefix:}")
# .format(**client.__dict__)

Domain: {domain:}
Session: {session:}
URI Prefix: {uri_prefix:}


In [6]:
#Identify all columns available in dataset

metadata = client.get_metadata(socrata_dataset_identifier)
[x['name'] for x in metadata['columns']]

['Plate',
 'State',
 'License Type',
 'Summons Number',
 'Issue Date',
 'Violation Time',
 'Violation',
 'Judgment Entry Date',
 'Fine Amount',
 'Penalty Amount',
 'Interest Amount',
 'Reduction Amount',
 'Payment Amount',
 'Amount Due',
 'Precinct',
 'County',
 'Issuing Agency',
 'Violation Status',
 'Summons Image']

In [7]:
#Pull API. Limit number is based on previously identified as the number of street parking violations from the total 50m rows of data

results = client.get(socrata_dataset_identifier,\
                     select=('Plate, State,Issue_Date,Violation_Time,Fine_Amount,Amount_Due,Payment_Amount,\
                     Precinct,County'),\
                     where ="(violation='NO PARKING-STREET CLEANING')",\
#                     limit=10000)
                     limit=6766400)
results_df = pd.DataFrame.from_dict(results)
results_df.head()

,Plate,State,Issue_Date,Violation_Time,Fine_Amount,Amount_Due,Payment_Amount,Precinct,County
0,HTE1892,NY,03/06/2020,09:13A,65,65,0,044,BX
1,556749R,NJ,03/06/2020,09:11A,65,65,0,044,BX
2,99940ML,NY,03/06/2020,09:09A,65,65,0,044,BX
3,99867ML,NY,03/06/2020,09:08A,65,65,0,044,BX
4,JNR5500,NY,03/06/2020,08:27A,65,65,0,044,BX


In [8]:
results_df.shape

(6766400, 9)

# Data Analysis

In [9]:
violation_df = results_df.copy()

## Data Cleaning

In [10]:
violation_df.head()

,Plate,State,Issue_Date,Violation_Time,Fine_Amount,Amount_Due,Payment_Amount,Precinct,County
0,HTE1892,NY,03/06/2020,09:13A,65,65,0,044,BX
1,556749R,NJ,03/06/2020,09:11A,65,65,0,044,BX
2,99940ML,NY,03/06/2020,09:09A,65,65,0,044,BX
3,99867ML,NY,03/06/2020,09:08A,65,65,0,044,BX
4,JNR5500,NY,03/06/2020,08:27A,65,65,0,044,BX


In [11]:
violation_df['time_issued'] = (violation_df['Issue_Date'] + ' ' + violation_df['Violation_Time'])
violation_df.head()

,Plate,State,Issue_Date,Violation_Time,Fine_Amount,Amount_Due,Payment_Amount,Precinct,County,time_issued
0,HTE1892,NY,03/06/2020,09:13A,65,65,0,044,BX,03/06/2020 09:13A
1,556749R,NJ,03/06/2020,09:11A,65,65,0,044,BX,03/06/2020 09:11A
2,99940ML,NY,03/06/2020,09:09A,65,65,0,044,BX,03/06/2020 09:09A
3,99867ML,NY,03/06/2020,09:08A,65,65,0,044,BX,03/06/2020 09:08A
4,JNR5500,NY,03/06/2020,08:27A,65,65,0,044,BX,03/06/2020 08:27A


In [ ]:
violation_df['time_issued']=pd.to_datetime(violation_df['time_issued'], errors='coerce')
violation_df.head(10)

In [ ]:
violation_df.drop(['Violation_Time', 'Issue_Date'], axis=1, inplace=True)
violation_df.loc[violation_df['time_issued']=='NaT', 'time_issued']= np.nan 
violation_df.dropna(inplace = True)

In [ ]:
violation_df['Precinct'] = violation_df['Precinct'].astype(float)
violation_df['Fine_Amount'] = violation_df['Fine_Amount'].astype(float)
violation_df['Amount_Due'] = violation_df['Amount_Due'].astype(float)
violation_df['Payment_Amount'] = violation_df['Amount_Due'].astype(float)

In [ ]:
violation_df.loc[violation_df['Precinct']>123, 'Precinct']= np.nan 
violation_df.dropna(inplace = True)

In [ ]:
violation_df.shape

In [ ]:
violation_df.info()

In [ ]:
violation_df.loc[violation_df['time_issued']>'2019-12-31 12:00:00', 'time_issued']= np.nan
violation_df.loc[violation_df['time_issued']<'2017-01-01 12:00:00', 'time_issued']= np.nan
violation_df.dropna(inplace = True)

In [ ]:
min(violation_df['time_issued'])

In [ ]:
max(violation_df['time_issued'])

In [ ]:
violation_df.shape

In [ ]:
# move cols

violation_df = violation_df[[ 'Plate','time_issued', 'Fine_Amount', 'Amount_Due', 'Payment_Amount',
       'Precinct', 'County', 'State']]
violation_df.head()

In [ ]:
violation_df['weekday'] = violation_df['time_issued'].dt.weekday
county_dummies = pd.get_dummies(violation_df['County'], prefix='County')
weekday_dummies = pd.get_dummies(violation_df['weekday'], prefix='weekday')
violation_df = pd.concat([violation_df, county_dummies, weekday_dummies], axis=1)
violation_df.drop(columns =['County','weekday'], inplace=True)
violation_df.columns

In [ ]:
violation_df.rename(columns={'County_BX':'Bronx', 'County_K':'Brooklyn', \
                             'County_NY':'Manhattan', 'County_Q':'Queens', \
                             'County_R':'Staten_Island'}, inplace=True)

In [ ]:
#Clean this up later

violation_df.loc[violation_df['State'] =='NY', 'State']= 1

out_of_state = {'FL':0, 'NJ':0, 'MI':0, 'RI':0, 'PA':0, 'VA':0, 'ME':0, 'CT':0, 'NC':0, 'AL':0,
       'TX':0, 'IL':0, 'MA':0, 'WI':0, 'IN':0, 'GV':0, 'NH':0, 'OH':0, '99':0, 'AZ':0, 'CA':0,
       'SC':0, 'LA':0, 'GA':0, 'VT':0, 'OR':0, 'MD':0, 'SD':0, 'MO':0, 'DE':0, 'WV':0, 'OK':0,
       'ON':0, 'CO':0, 'AR':0, 'AK':0, 'QB':0, 'WA':0, 'MT':0, 'NV':0, 'UT':0, 'TN':0, 'DC':0, 'YT':0,
       'NE':0, 'KY':0, 'IA':0, 'MN':0, 'ID':0, 'KS':0, 'ND':0, 'MS':0, 'NM':0, 'WY':0, 'HI':0, 'NF':0,
       'MX':0, 'DP':0, 'AB':0, 'NS':0, 'BC':0, 'PR':0, 'NB':0, 'FO':0, 'MB':0, 'PE':0, 'SK':0, 'NT':0}

violation_df['State'].replace(out_of_state, inplace = True)
violation_df['State'].value_counts()

In [ ]:
violation_df.rename(columns={'Precinct':'precinct'}, inplace=True)
violation_df.head()

In [ ]:
zero_dict = {0: np.nan}
violation_df['precinct'].replace(zero_dict, inplace = True)
violation_df.dropna(inplace = True)

violation_df.info()

In [ ]:
sns.heatmap(violation_df.isnull(), cbar=False)

In [ ]:
violation_df.head()

In [ ]:
violation_df.to_csv('ParkingTickets_Data_Raw.csv')